### 1. Imports and Setup

In [1]:
import pandas as pd
import sqlite3
import requests

In [2]:
# Load SQL magic
%load_ext sql
%sql sqlite:///finance.db
%config SqlMagic.style = '_DEPRECATED_default'

### 2. Scrape Financial Data
We use Yahoo Finance Most Active Stocks.

In [3]:
headers = {
	"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
url = "https://finance.yahoo.com/most-active"
response = requests.get(url, headers=headers)
tables = pd.read_html(response.text)
stocks_df = tables[0]
stocks_df.head()

C:\Users\nitai\AppData\Local\Temp\ipykernel_124632\2067141439.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


,Symbol,Name,Unnamed: 2,Price,Change,Change %,Volume,Avg Vol (3M),Market Cap,P/E Ratio (TTM),52 Wk Change %,52 Wk Range
0,NVDA,NVIDIA Corporation,NaN,170.70 +6.63 (+4.04%),6.63,+4.04%,228.639M,207.49M,4.163T,54.89,+44.67%,NaN
1,NU,Nu Holdings Ltd.,NaN,13.54 +0.47 (+3.60%),0.47,+3.60%,118.407M,63.18M,65.323B,30.77,+2.42%,NaN
2,WBD,"Warner Bros. Discovery, Inc.",NaN,12.03 +0.02 (+0.17%),0.02,+0.17%,94.497M,48.604M,29.763B,--,+44.59%,NaN
3,LCID,"Lucid Group, Inc.",NaN,2.3300 +0.0600 (+2.64%),0.06,+2.64%,90.179M,128.47M,7.107B,--,-38.52%,NaN
4,AMD,"Advanced Micro Devices, Inc.",NaN,155.61 +9.37 (+6.41%),9.37,+6.41%,91.679M,45.945M,252.306B,113.58,-2.40%,NaN


### 3. Create Database & Table

In [4]:
conn = sqlite3.connect('finance.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS stocks (
        Symbol TEXT,
        Name TEXT,
        Price REAL,
        Change REAL,
        Percent_Change TEXT,
        Volume INTEGER,
        Avg_Volume INTEGER,
        Market_Cap TEXT
    )
''')
conn.commit()

### 4. Insert Scraped Data into Table
Map columns to SQL table columns.

In [5]:
stocks_df = stocks_df.rename(columns={
    'Symbol': 'Symbol',
    'Name': 'Name',
    'Price (Intraday)': 'Price',
    'Change': 'Change',
    'Change %': 'Percent_Change',
    'Volume': 'Volume',
    'Avg Vol (3M)': 'Avg_Volume',
    'Market Cap': 'Market_Cap'
})

# Convert 'Volume' and 'Avg_Volume' columns to integers
def convert_to_int(value):
    if pd.isna(value) or value == '-':
        return 0
    value_str = str(value).strip()
    if value_str.endswith('M'):
        return int(float(value_str[:-1]) * 1_000_000)
    elif value_str.endswith('K'):
        return int(float(value_str[:-1]) * 1_000)
    elif value_str.replace(',', '').replace('.', '', 1).isdigit():
        return int(float(value_str))
    else:
        return 0

stocks_df['Volume'] = stocks_df['Volume'].apply(convert_to_int)
stocks_df['Avg_Volume'] = stocks_df['Avg_Volume'].apply(convert_to_int)
stocks_df['Price'] = stocks_df['Price'].str.split(' ').str[0].astype(float)

stocks_df.to_sql('stocks', conn, if_exists='replace', index=False)

26

## 5. SQL Queries

### A) WHERE

In [6]:
%%sql
SELECT *
FROM stocks 
WHERE Price > 100;

 * sqlite:///finance.db
Done.


Symbol,Name,Unnamed: 2,Price,Change,Percent_Change,Volume,Avg_Volume,Market_Cap,P/E Ratio (TTM),52 Wk Change %,52 Wk Range
NVDA,NVIDIA Corporation,None,170.7,6.63,+4.04%,228639000,207490000,4.163T,54.89,+44.67%,None
AMD,"Advanced Micro Devices, Inc.",None,155.61,9.37,+6.41%,91679000,45945000,252.306B,113.58,-2.40%,None
TSLA,"Tesla, Inc.",None,310.78,-6.12,-1.93%,76805000,111667000,1.001T,178.61,+25.06%,None
PLTR,Palantir Technologies Inc.,None,148.58,-0.57,-0.38%,58852000,88877000,350.635B,675.36,+426.51%,None


### B) GROUP BY

In [7]:
%%sql
SELECT Market_Cap, COUNT(*) as stock_count FROM stocks GROUP BY Market_Cap;

 * sqlite:///finance.db
Done.


Market_Cap,stock_count
None,1
1.001T,1
11.739B,1
139.917B,1
23.168B,1
24.161B,1
24.745B,1
252.306B,1
253.958B,1
29.763B,1


### C) ORDER BY

In [8]:
%%sql
SELECT * FROM stocks ORDER BY Price DESC;

 * sqlite:///finance.db
Done.


Symbol,Name,Unnamed: 2,Price,Change,Percent_Change,Volume,Avg_Volume,Market_Cap,P/E Ratio (TTM),52 Wk Change %,52 Wk Range
TSLA,"Tesla, Inc.",None,310.78,-6.12,-1.93%,76805000,111667000,1.001T,178.61,+25.06%,None
NVDA,NVIDIA Corporation,None,170.7,6.63,+4.04%,228639000,207490000,4.163T,54.89,+44.67%,None
AMD,"Advanced Micro Devices, Inc.",None,155.61,9.37,+6.41%,91679000,45945000,252.306B,113.58,-2.40%,None
PLTR,Palantir Technologies Inc.,None,148.58,-0.57,-0.38%,58852000,88877000,350.635B,675.36,+426.51%,None
HOOD,"Robinhood Markets, Inc.",None,99.54,-0.42,-0.42%,45646000,43777000,87.836B,56.88,+311.66%,None
TTD,"The Trade Desk, Inc.",None,80.4,4.97,+6.59%,43299000,9177000,39.512B,98.05,-15.58%,None
WFC,Wells Fargo & Company,None,78.86,-4.57,-5.48%,53411000,17321000,253.958B,13.55,+29.47%,None
MP,MP Materials Corp.,None,58.22,9.7,+19.99%,70760000,12431000,9.516B,--,+285.31%,None
SMCI,"Super Micro Computer, Inc.",None,53.17,3.44,+6.92%,61111000,46290000,31.733B,28.90,-34.87%,None
BAC,Bank of America Corporation,None,46.15,-0.92,-1.95%,46412000,40336000,347.596B,13.78,+4.93%,None


### D) LIMIT

In [9]:
%%sql
SELECT * FROM stocks LIMIT 5;

 * sqlite:///finance.db
Done.


Symbol,Name,Unnamed: 2,Price,Change,Percent_Change,Volume,Avg_Volume,Market_Cap,P/E Ratio (TTM),52 Wk Change %,52 Wk Range
NVDA,NVIDIA Corporation,None,170.7,6.63,+4.04%,228639000,207490000,4.163T,54.89,+44.67%,None
NU,Nu Holdings Ltd.,None,13.54,0.47,+3.60%,118407000,63180000,65.323B,30.77,+2.42%,None
WBD,"Warner Bros. Discovery, Inc.",None,12.03,0.02,+0.17%,94497000,48604000,29.763B,--,+44.59%,None
LCID,"Lucid Group, Inc.",None,2.33,0.06,+2.64%,90179000,128470000,7.107B,--,-38.52%,None
AMD,"Advanced Micro Devices, Inc.",None,155.61,9.37,+6.41%,91679000,45945000,252.306B,113.58,-2.40%,None


### E) AS

In [10]:
%%sql
SELECT Symbol AS Ticker, Name AS Company_Name, Price AS Stock_Price, Market_Cap FROM stocks;

 * sqlite:///finance.db
Done.


Ticker,Company_Name,Stock_Price,Market_Cap
NVDA,NVIDIA Corporation,170.7,4.163T
NU,Nu Holdings Ltd.,13.54,65.323B
WBD,"Warner Bros. Discovery, Inc.",12.03,29.763B
LCID,"Lucid Group, Inc.",2.33,7.107B
AMD,"Advanced Micro Devices, Inc.",155.61,252.306B
QS,QuantumScape Corporation,10.85,6.064B
TSLA,"Tesla, Inc.",310.78,1.001T
ERIC,Telefonaktiebolaget LM Ericsson (publ),7.22,24.745B
MP,MP Materials Corp.,58.22,9.516B
NIO,NIO Inc.,4.25,9.613B


### F) HAVING

In [11]:
%%sql
SELECT Name, Market_Cap, AVG(Price) as avg_price FROM stocks GROUP BY Market_Cap HAVING avg_price > 100;

 * sqlite:///finance.db
Done.


Name,Market_Cap,avg_price
"Tesla, Inc.",1.001T,310.78
"Advanced Micro Devices, Inc.",252.306B,155.61
Palantir Technologies Inc.,350.635B,148.58
NVIDIA Corporation,4.163T,170.7


### G) BETWEEN AND

In [12]:
%%sql
SELECT * FROM stocks WHERE Price BETWEEN 200 AND 350;

 * sqlite:///finance.db
Done.


Symbol,Name,Unnamed: 2,Price,Change,Percent_Change,Volume,Avg_Volume,Market_Cap,P/E Ratio (TTM),52 Wk Change %,52 Wk Range
TSLA,"Tesla, Inc.",None,310.78,-6.12,-1.93%,76805000,111667000,1.001T,178.61,+25.06%,None


### H) AVG, SUM, COUNT

In [13]:
%%sql
SELECT AVG(Price) AS avg_price, SUM(Price) AS total_price, COUNT(*) AS count_stocks FROM stocks;

 * sqlite:///finance.db
Done.


avg_price,total_price,count_stocks
55.836800000000004,1395.92,26


### I) DISTINCT, UPDATE, INSERT

In [14]:
%%sql
SELECT DISTINCT Market_Cap FROM stocks;

 * sqlite:///finance.db
Done.


Market_Cap
4.163T
65.323B
29.763B
7.107B
252.306B
6.064B
1.001T
24.745B
9.516B
9.613B


In [15]:
%%sql
UPDATE stocks SET Price = Price + 1 WHERE Symbol = 'AAPL';

 * sqlite:///finance.db
0 rows affected.


[]

In [16]:
%%sql
INSERT INTO stocks (Symbol, Name, Price, Change, Percent_Change, Volume, Avg_Volume, Market_Cap)
VALUES ('TEST', 'Test Company', 123.45, 0.5, '+0.4%', 10000, 5000, 'Small Cap');

 * sqlite:///finance.db
1 rows affected.


[]

### J) LIKE

In [17]:
%%sql
SELECT * FROM stocks WHERE Name LIKE '%Inc%';

 * sqlite:///finance.db
Done.


Symbol,Name,Unnamed: 2,Price,Change,Percent_Change,Volume,Avg_Volume,Market_Cap,P/E Ratio (TTM),52 Wk Change %,52 Wk Range
WBD,"Warner Bros. Discovery, Inc.",None,12.03,0.02,+0.17%,94497000,48604000,29.763B,--,+44.59%,None
LCID,"Lucid Group, Inc.",None,2.33,0.06,+2.64%,90179000,128470000,7.107B,--,-38.52%,None
AMD,"Advanced Micro Devices, Inc.",None,155.61,9.37,+6.41%,91679000,45945000,252.306B,113.58,-2.40%,None
TSLA,"Tesla, Inc.",None,310.78,-6.12,-1.93%,76805000,111667000,1.001T,178.61,+25.06%,None
NIO,NIO Inc.,None,4.25,0.08,+1.92%,65803000,38148000,9.613B,--,-7.41%,None
HBAN,Huntington Bancshares Incorporated,None,16.56,-0.51,-2.99%,63638000,25589000,24.161B,12.74,+14.68%,None
ACHR,Archer Aviation Inc.,None,11.26,0.46,+4.26%,63447000,36112000,6.182B,--,+122.09%,None
SOFI,"SoFi Technologies, Inc.",None,20.96,-0.37,-1.73%,60283000,69153000,23.168B,48.74,+167.35%,None
SMCI,"Super Micro Computer, Inc.",None,53.17,3.44,+6.92%,61111000,46290000,31.733B,28.90,-34.87%,None
AAL,American Airlines Group Inc.,None,12.17,-0.21,-1.70%,58353000,59639000,8.026B,12.17,+10.54%,None
